<a href="https://colab.research.google.com/github/creativesar/Hangman-project/blob/main/hangman_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import string

def get_word():
    words = ['python', 'programming', 'computer', 'algorithm', 'developer',
             'keyboard', 'challenge', 'learning', 'hangman', 'solution']
    return random.choice(words).upper()

def hangman():
    word = get_word()
    word_letters = set(word)  # letters in the word
    alphabet = set(string.ascii_uppercase)
    used_letters = set()  # what the user has guessed

    lives = 6  # Number of lives

    # Getting user input
    while len(word_letters) > 0 and lives > 0:
        # Letters used
        print('You have', lives, 'lives left and you have used these letters: ', ' '.join(used_letters))

        # What current word is (ie W - R D)
        word_list = [letter if letter in used_letters else '-' for letter in word]
        print('Current word: ', ' '.join(word_list))

        user_letter = input('Guess a letter: ').upper()
        if user_letter in alphabet - used_letters:
            used_letters.add(user_letter)
            if user_letter in word_letters:
                word_letters.remove(user_letter)
                print('')
            else:
                lives = lives - 1  # takes away a life if wrong
                print('\nYour letter,', user_letter, 'is not in the word.')
        elif user_letter in used_letters:
            print('\nYou have already used that letter. Guess another letter.')
        else:
            print('\nThat is not a valid letter.')

    # gets here when len(word_letters) == 0 OR when lives == 0
    if lives == 0:
        print('You died, sorry. The word was', word)
    else:
        print('Congratulations! You guessed the word', word, '!!')

if __name__ == "__main__":
    print("Welcome to Hangman!")
    hangman()
    while input("Play Again? (Y/N) ").upper() == "Y":
        hangman()

Welcome to Hangman!
You have 6 lives left and you have used these letters:  
Current word:  - - - - - - - - - - -
